In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score
from medmnist import INFO
from medmnist.dataset import ChestMNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from transformers import ViTForImageClassification, ViTConfig

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Dataset preparation
dataset_name = "chestmnist"
info = INFO[dataset_name]
num_classes = len(info['label'])
print(f"Dataset: {dataset_name}, Num Classes: {num_classes}")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_data = ChestMNIST(split='train', transform=transform, download=True)
test_data = ChestMNIST(split='test', transform=transform, download=True)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


vgg16_model = models.vgg16(pretrained=True)
vgg16_model.classifier[6] = nn.Linear(vgg16_model.classifier[6].in_features, num_classes)

resnet50_model = models.resnet50(pretrained=True)
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)

resnet18_model = models.resnet18(pretrained=True)
resnet18_model.fc = nn.Linear(resnet18_model.fc.in_features, num_classes)

densenet_model = models.densenet121(pretrained=True)
densenet_model.classifier = nn.Linear(densenet_model.classifier.in_features, num_classes)

vit_config = ViTConfig.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k', 
    num_labels=num_classes
).to(device)

vgg16_model, resnet50_model, resnet18_model, densenet_model = [
    model.to(device) for model in [vgg16_model, resnet50_model, resnet18_model, densenet_model]
]


criterion = nn.BCEWithLogitsLoss()


def extract_vit_outputs(outputs):
    return outputs.logits if hasattr(outputs, "logits") else outputs


def calculate_accuracy(outputs, targets):
    preds = torch.sigmoid(outputs) >= 0.5
    correct = (preds == targets).sum().float()
    accuracy = correct / targets.numel()
    return accuracy


def train_model(model, optimizer, criterion, scheduler, num_epochs=10, model_name=""):
    results = {"train_acc": [], "test_acc": [], "train_loss": [], "test_loss": [], "test_auc": []}
    
    for epoch in range(num_epochs):
        
        model.train()
        train_loss, train_accuracy = 0.0, 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device).float()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = extract_vit_outputs(outputs)  # Handle ViT outputs
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_accuracy += calculate_accuracy(outputs, targets).item()
        
        results["train_loss"].append(train_loss / len(train_loader))
        results["train_acc"].append(train_accuracy / len(train_loader))
        
        
        model.eval()
        test_loss, test_accuracy = 0.0, 0.0
        all_targets, all_preds = [], []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device).float()
                outputs = model(inputs)
                outputs = extract_vit_outputs(outputs)  # Handle ViT outputs
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                
                test_accuracy += calculate_accuracy(outputs, targets).item()
                all_preds.append(torch.sigmoid(outputs).cpu())
                all_targets.append(targets.cpu())
        
        results["test_loss"].append(test_loss / len(test_loader))
        results["test_acc"].append(test_accuracy / len(test_loader))
        
       
        all_preds = torch.cat(all_preds).numpy()
        all_targets = torch.cat(all_targets).numpy()
        auc = roc_auc_score(all_targets, all_preds, average="macro")
        results["test_auc"].append(auc)
        
        scheduler.step()
        print(
            f"[{model_name}] Epoch {epoch+1}/{num_epochs} | "
            f"Train Acc: {results['train_acc'][-1]*100:.2f}% | "
            f"Test Acc: {results['test_acc'][-1]*100:.2f}% | "
            f"AUC: {auc:.4f}"
        )
    
    os.makedirs("results", exist_ok=True)
    with open(f"results/{model_name}_results.json", "w") as f:
        json.dump(results, f)
    return results


optimizers = {
    "VGG16": optim.Adam(vgg16_model.parameters(), lr=1e-4),
    "ResNet50": optim.Adam(resnet50_model.parameters(), lr=1e-4),
    "ResNet18": optim.Adam(resnet18_model.parameters(), lr=1e-4),
    "DenseNet": optim.Adam(densenet_model.parameters(), lr=1e-4),
    "ViT": optim.Adam(vit_model.parameters(), lr=1e-4)
}

schedulers = {
    model_name: lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    for model_name, optimizer in optimizers.items()
}

# Train All Models
models_dict = {
    "VGG16": vgg16_model,
    "ResNet50": resnet50_model,
    "ResNet18": resnet18_model,
    "DenseNet": densenet_model,
    "ViT": vit_model
}
results = {}

for model_name, model in models_dict.items():
    print(f"\nTraining {model_name}...")
    try:
        results[model_name] = train_model(
            model, optimizers[model_name], criterion, schedulers[model_name], num_epochs=10, model_name=model_name
        )
    except Exception as e:
        print(f"Error occurred while training {model_name}: {e}")
        continue

# Visualization
def plot_results(results, metric, ylabel):
    plt.figure(figsize=(10, 6))
    for model_name, result in results.items():
        if metric in result:
            plt.plot(result[metric], label=model_name)
    plt.xlabel("Epochs")
    plt.ylabel(ylabel)
    plt.title(f'{ylabel} vs Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot Results
metrics = {"train_acc": "Training Accuracy", "test_acc": "Testing Accuracy",
           "train_loss": "Training Loss", "test_loss": "Testing Loss", "test_auc": "Testing AUC"}

for metric, ylabel in metrics.items():
    plot_results(results, metric, ylabel)


Dataset: chestmnist, Num Classes: 14


100%|██████████| 82802576/82802576 [00:07<00:00, 11709328.92it/s]


Using downloaded and verified file: /root/.medmnist/chestmnist.npz


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 62.8MB/s] 
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training VGG16...
[VGG16] Epoch 1/10 | Train Acc: 94.84% | Test Acc: 94.75% | AUC: 0.7293
[VGG16] Epoch 2/10 | Train Acc: 94.90% | Test Acc: 94.80% | AUC: 0.7606
[VGG16] Epoch 3/10 | Train Acc: 94.92% | Test Acc: 94.78% | AUC: 0.7589
[VGG16] Epoch 4/10 | Train Acc: 94.96% | Test Acc: 94.76% | AUC: 0.7760
[VGG16] Epoch 5/10 | Train Acc: 95.03% | Test Acc: 94.78% | AUC: 0.7725
[VGG16] Epoch 6/10 | Train Acc: 95.31% | Test Acc: 94.65% | AUC: 0.7743
[VGG16] Epoch 7/10 | Train Acc: 95.48% | Test Acc: 94.60% | AUC: 0.7700
[VGG16] Epoch 8/10 | Train Acc: 95.64% | Test Acc: 94.49% | AUC: 0.7608
[VGG16] Epoch 9/10 | Train Acc: 95.83% | Test Acc: 94.39% | AUC: 0.7541
[VGG16] Epoch 10/10 | Train Acc: 96.09% | Test Acc: 94.30% | AUC: 0.7448

Training ResNet50...
[ResNet50] Epoch 1/10 | Train Acc: 94.84% | Test Acc: 94.76% | AUC: 0.7694
[ResNet50] Epoch 2/10 | Train Acc: 94.91% | Test Acc: 94.80% | AUC: 0.7827
[ResNet50] Epoch 3/10 | Train Acc: 94.95% | Test Acc: 94.80% | AUC: 0.7888
[ResNet50] Ep

In [1]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=d2dc4e6fbf853bb10723d60c009e8aa9ea47325e46c68f6886ba4bab50652989
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
